# 拉普拉斯方法定轨道六根数

## 1. 已知三次观测的观测坐标，定出某一时刻的位置矢量和速度矢量
按照笔记的方法分别推出各个量

注意时间单位的选取!

In [2]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import datetime
import time

from DDS_Laplace import *

import DDS_CMpack as DDS

%matplotlib inline

## PART 1 星表的format

In [3]:
# astronomy constants
R_earth = 6371
au = 149597870
r_station_earth = 0.999102 #*R_earth
r_earth_sun = 1*au

time_unit = 806.81163 #806.8116 # SI 


In [4]:
# give 3 observed position
# position_fram = pd.DataFrame({'alpha':\
#                         np.array([142.935,157.274166667,171.817916667,183.134166667,\
#                                   208.640416667,219.102083333]),\
#                        'delta':\
#                        np.array([8.521111111,-2.395,-14.508333333,-23.56333333,-40.020277777,-44.998611111]),\
#                        'UT1_hours':np.array([21.575128333,21.60311055,21.631419722,\
#                                              21.654491388,21.714094166,21.742875555])})

position_fram = pd.read_csv('Echo1st.csv')

# position_fram

In [5]:
# 计算观测数据的儒略日时间和恒星时
year = 1965; month = 1; days = 14;
UT1_hours = np.array([21.575128333,21.60311055,21.631419722,21.654491388,21.714094166,21.742875555])
jd_t = []
SG_t = []
for i in range(len(UT1_hours)):
    SG_s,jd_s = DDS.UTC2SG(year,month,days,UT1_hours[i])
    SG_t.append(SG_s); jd_t.append(jd_s);
    
SG_array = np.array(SG_t); 
SG_degree = SG_array%(3600*24)/(3600*24)*360

jd_array = np.array(jd_t)
delta_t = (UT1_hours - UT1_hours[0])*3600/time_unit
delta_t_SI = delta_t * time_unit
position_fram['deltaT'] = delta_t

delta_t

array([0.        , 0.12485688, 0.25117263, 0.35411859, 0.62006667,
       0.74848946])

In [6]:
# give the station's lam and phi
lam = 118.82091666
phi = 31.893611111


In [7]:
# 需要根据alpha和delta求出所需的测站到地心的RA,DEC
station_earth = []
for i in range(len(position_fram)):
    # give the station's RA and DEC then convert to station_earth
    RA_station = SG_degree[i] + lam
    DEC_station = phi
    station_earth.append(DDS.RADEC2xyz(r_station_earth,RA_station,DEC_station))

station_earth

[array([-0.81183633, -0.24592818,  0.52786921]),
 array([-0.81000791, -0.25188507,  0.52786921]),
 array([-0.80811365, -0.25789773,  0.52786921]),
 array([-0.80653683, -0.26278746,  0.52786921]),
 array([-0.80232652, -0.27537441,  0.52786921]),
 array([-0.80022302, -0.28142853,  0.52786921])]

In [8]:
# position_fram

In [9]:
# add Pj ,Qj to positon_fram_fil
Lt = []; Pt = []; Qt = []
Lambda = []; Miu = []; Niu = [];

#由 delta,alpha 算出 L = [Lambda,Miu,Niu]
for j in range(len(position_fram)):
    delta = position_fram['delta'].values[j]*2*np.pi/360 ; 
    alpha = position_fram['alpha'].values[j]*2*np.pi/360;

    L = [np.cos(delta)*np.cos(alpha),\
         np.cos(delta)*np.sin(alpha),\
         np.sin(delta)]
    
    Lambda.append(L[0]); 
    Miu.append(L[1]);
    Niu.append(L[2]);
    
#     Lambda.append(np.cos(delta)*np.cos(alpha))
#     Miu.append(np.cos(delta)*np.sin(alpha))
#     Niu.append(np.sin(delta))

# 由[Lambda,Miu,Niu] 和 测站-地心矢量归算出 P,Q
    Pj = L[2]*station_earth[j][0] - L[0]*station_earth[j][2]
    Qj = L[2]*station_earth[j][1] - L[1]*station_earth[j][2]
    
    Lt.append(L)
    Pt.append(Pj)
    Qt.append(Qj)
#     print(Lt,Pt,Qt)

position_fram_New = pd.DataFrame({'jd':jd_array,
                                 'SG_d':SG_degree,
                                 'P':np.array(Pt),'Q':np.array(Qt),
                                  'deltaT_SI':delta_t_SI,
                                 'Lambda':np.array(Lambda),
                                 'Miu':np.array(Miu),'Niu':np.array(Niu)})

Lt 

[[-0.7891439307993768, 0.5960674272555554, 0.14817381228628504],
 [-0.9215582970478929, 0.38598449379160416, -0.04178846365477428],
 [-0.9582566000732619, 0.13778102486286736, -0.25052081271577153],
 [-0.9152477093203553, -0.0501154444748697, -0.39976252051552885],
 [-0.6721154419353875, -0.36706452095992986, -0.6430586832976101],
 [-0.5487447629378402, -0.44598590319844417, -0.7070896402130082]]

In [10]:
position_fram_fil = pd.concat([position_fram,position_fram_New],axis=1)

In [11]:
position_fram_fil.to_csv('echo1st_format.csv')
position_fram_fil

,UT1_hours,alpha,delta,year,month,days,deltaT,Lambda,Miu,Niu,P,Q,SG_d,deltaT_SI,jd
0,21.575128,142.935000,8.521111,1965,1,14,0.000000,-0.789144,0.596067,0.148174,0.296272,-0.351086,78.032128,0.000000,-12769.600293
1,21.603111,157.274167,-2.395000,1965,1,14,0.124857,-0.921558,0.385984,-0.041788,0.520311,-0.193223,78.453011,100.735981,-12769.599128
2,21.631420,171.817917,-14.508333,1965,1,14,0.251173,-0.958257,0.137781,-0.250521,0.708283,-0.008122,78.878811,202.649000,-12769.597948
3,21.654491,183.134167,-23.563333,1965,1,14,0.354119,-0.915248,-0.050115,-0.399763,0.805554,0.131507,79.225833,285.706998,-12769.596987
4,21.714094,208.640417,-40.020278,1965,1,14,0.620067,-0.672115,-0.367065,-0.643059,0.870732,0.370844,80.122323,500.276999,-12769.594503
5,21.742876,219.102083,-44.998611,1965,1,14,0.748489,-0.548745,-0.445986,-0.707090,0.855495,0.434417,80.555226,603.889999,-12769.593304


## PART 2 给出t0,算出t0的F,G,进而得出r0,v0

In [12]:
t0 = 0.5*(position_fram_fil['deltaT'].values[0] + position_fram_fil['deltaT'].values[5])
observed_choose = [1,3,5]





r0_a,v0_a = Calculatefort0(position_fram_fil,observed_choose,t0)

0 : 0.9984226162730335 0.04800349905540134
[0.03194931 0.0002037  0.07076899] [2.70966570e-03 1.36737629e-06 9.03982771e-03]
--------------------
1 : 1.5139389870973732 1.7156830515699655
[0.01971213 0.00014397 0.05538079] [1.53640244e-03 9.62029085e-07 7.26806261e-03]
--------------------
2 : 1.1180455769847804 0.5119898679876053
[1.30097450e-02 8.65400549e-05 2.84405528e-02] [1.06962099e-03 5.80545151e-07 3.45320468e-03]
--------------------
3 : 1.3277824392203847 1.189450828058739
[8.27294713e-03 5.81603520e-05 2.02816459e-02] [6.57554048e-04 3.89386192e-07 2.52542070e-03]
--------------------
4 : 1.1803973151079028 0.7278185547910484
[5.40724101e-03 3.65849407e-05 1.20782509e-02] [4.39650820e-04 2.45306250e-07 1.46361747e-03]
--------------------
5 : 1.269370389075524 1.0128768484239639
[3.47267799e-03 2.40544923e-05 8.16717117e-03] [2.78365795e-04 1.61147940e-07 1.00259023e-03]
--------------------
6 : 1.2096396785263392 0.8240466312977627
[2.25552476e-03 1.53805417e-05 5.11341846

In [13]:
r0 = np.sqrt(np.dot(r0_a,r0_a))
v0 = np.sqrt(np.dot(v0_a,v0_a))

print(r0_a*R_earth,'r0:',r0*R_earth,'km;','h:',(r0-r_station_earth)*R_earth,'km')

print(v0_a*R_earth/time_unit,'v0:',v0*R_earth/time_unit,'km/s')

print(t0*time_unit,'s')

[-7245.98540944 -1874.17826161  2365.63579833] r0: 7849.400068826888 km; h: 1484.1212268268876 km
[-0.05590008 -5.20070644 -4.79699883] v0: 7.07542720284154 km/s
301.94499960000485 s


In [14]:
r0_a*R_earth

array([-7245.98540944, -1874.17826161,  2365.63579833])

In [15]:
v0_a*R_earth/time_unit

array([-0.05590008, -5.20070644, -4.79699883])

## 2. 已知某一时刻的位置矢量和速度矢量，求轨道六根数

按照笔记方法,逐步求得：
* a
* n()
* E()
* e
* M
* i Omega w

In [16]:
# 从第二步开始，因为不需要用到F,G的计算，所以不用再取理论单位，现在取国际单位制
# astronomy constants
R_earth = 6371e3 # m

M_earth = 5.965e24 # kg
G_graviation = 6.672e-11 # N·m^2 /kg^2 

au = 149597870e3 # m
r_station_earth = 0.999102 #*R_earth
r_earth_sun = 1*au

time_unit = 806.81163 #806.8116 # SI 

# miu_GM = 398600.5e-6 # km^3/SI^2

miu_GM = G_graviation*M_earth # m^3/SI^2

In [17]:
a,e,M,i,Omega,w,P,Q,R = Calculatefor6elements(r0_a,v0_a,t0)


a 7751418.3357464885 m
n: 0.0009244032824160432 rad/s - Not Output
e: 0.02496681410596837 Nan
E: 4.181509802684073 rad - Not Output
M: 4.203040241507634 rad
i,Omega,w: 0.827385156755234 0.5479439050679501 -1.4400860495680883 rad/rad/rad


In [18]:
np.degrees(np.array([M,i,Omega,w]))

array([240.81646696,  47.40567751,  31.39487317, -82.51085278])

## 3. 由轨道6根数归算星历表

已知：
* a, # m
* n(), # rad/s
* E(), # rad
* e, # Nan
* M, # rad
* [i, Omega, w] = P,Q # [rad,rad,rad]

t0 = 121.0217445 **# unit = SI from the first data we have**

M = M0 + n(t-t0)

M = E - esin(E)

In [19]:
print('a',a,'e',e,'M',M,i,Omega,w)

a 7751418.3357464885 e 0.02496681410596837 M 4.203040241507634 0.827385156755234 0.5479439050679501 -1.4400860495680883


In [20]:
# 假设我们已知a,e,M,i,Omega,w,那么E,n均需要从六根数推出来。

n = np.sqrt(miu_GM/a**3)

n

0.0009244032824160432

In [21]:
# calculate keplar equation
# 设置初值t0 和我们需要归算的时间t

# 我将计算以整个周期为倍数的时间序列
# set T

T = 2*np.pi/n
print(T)


6797.017521138283


In [22]:
# set t

# t = np.arange(-201,T,) # unit = SI

t = (position_fram_fil['deltaT'].values - t0)*time_unit

t_forecast = t
Mt = (M + n*(t))%(2*np.pi)

print(t)
print(Mt)

[-301.9449996 -201.2090184  -99.2959992  -16.2380016  198.3319992
  301.9449996]
[3.92392129 4.01704196 4.11125069 4.18802978 4.38637899 4.48215919]


In [23]:
# 迭代 开普勒方程解 E
Et = []
epsilon = 1e-6
for M_single in Mt:
#     print(M_single)
    key = 0
    En0 = M_single
    Enw = [En0]
    Nwind = 0
    while key == 0:
        Enw.append(M_single + e*np.sin(Enw[Nwind]))

        Nwind += 1
#         print(Enw[Nwind])

        delta_Enw = np.abs(Enw[Nwind] - Enw[Nwind - 1])
#         print(delta_Enw)
#         print('------------')

        if delta_Enw < epsilon:
            key = 1;

    E_single = Enw[Nwind]
    print(E_single)
    Et.append(E_single)
# print('E:',Et)

3.906630176305218
3.9981768034472953
4.090951614954833
4.166689040743233
4.362921294943734
4.457995897105813


In [24]:
# 给出P_new, Q_new
P_new = np.array([np.cos(Omega)*np.cos(w) - np.sin(Omega)*np.sin(w)*np.cos(i),\
                 np.sin(Omega)*np.cos(w) + np.cos(Omega)*np.sin(w)*np.cos(i),\
                 np.sin(w)*np.sin(i) ])
Q_new = np.array([-np.cos(Omega)*np.sin(w) - np.sin(Omega)*np.cos(w)*np.cos(i),\
                 -np.sin(Omega)*np.sin(w) + np.cos(Omega)*np.cos(w)*np.cos(i),\
                 np.cos(w)*np.sin(i)])

print(P,Q)
print(P_new,Q_new)

[ 0.46064345 -0.50502068 -0.72990528] [0.79996673 0.59154962 0.0959532 ]
[ 0.46081815 -0.50489153 -0.72988436] [0.80036256 0.59178821 0.09595045]


In [25]:
# 归算rt, vt
rt = [];vt=[];
for Et_single in Et:
    rt_single = a*(np.cos(Et_single) - e)*P_new + a*np.sqrt(1 - e**2)*np.sin(Et_single)*Q_new
    
    rt_single_norm = np.sqrt( np.dot(rt_single,rt_single) )
    rt.append(rt_single)
    
    vt_single = -a**2*n/rt_single_norm*np.sin(Et_single)*P_new \
    + a**2*n/rt_single_norm*np.sqrt(1 - e**2)*np.cos(Et_single)*Q_new
    
    vt_single_norm = np.sqrt( np.dot(vt_single,vt_single) )
    vt.append(rt_single)
    
#     print(rt_single_norm/10**3,vt_single_norm/10**3)
    
rt = np.array(rt)
vt = np.array(vt)

In [26]:
# 计算预测的方向矢量L，赤经，赤纬
P_earth = rt
station_earth = np.array([-0.81000791, -0.25188507,  0.52786921])*R_earth

P_station = P_earth - station_earth

L_forecast_total = [];
Alpha_forecast = [];
Delta_forecast = [];
Nwind = 0
for P_station_single in P_station:
    # 计算我们测站的赤经Alpha_forecast 和赤纬Delta_forecast
    L_forecast_total.append( P_station_single/np.sqrt(np.dot(P_station_single,P_station_single)) )
    
    Delta_forecast.append( np.arcsin(L_forecast_total[Nwind][2]) )
    Alpha_forecast.append( arctan2(L_forecast_total[Nwind][1]/np.cos(Delta_forecast[Nwind]),\
                                   L_forecast_total[Nwind][0]/np.cos(Delta_forecast[Nwind])) )
    
    Nwind += 1;

##############################################

Forecast_fram = pd.DataFrame({'Alpha': np.array(Alpha_forecast),
                              'Alpha_d':np.degrees(np.array(Alpha_forecast)),
                             'Delta':np.array(Delta_forecast),
                              'Delta_d':np.degrees(np.array(Delta_forecast)),
                             'deltaT':t_forecast-t_forecast[0],
                             'x':rt[:,0],
                             'y':rt[:,1],
                             'z':rt[:,2],
                             'vx':vt[:,0],
                             'vy':vt[:,1],
                             'vz':vt[:,2]})

# print(L_forecast_total)

In [27]:
Forecast_fram.to_csv('echo1st_forecast.csv')
Forecast_fram


,Alpha,Alpha_d,Delta,Delta_d,deltaT,vx,vy,vz,x,y,z
0,2.498373,143.146252,0.151886,8.702443,0.000000,-6.961152e+06,-2.551088e+05,3.707492e+06,-6.961152e+06,-2.551088e+05,3.707492e+06
1,2.752704,157.718321,-0.036782,-2.107448,100.735981,-7.115216e+06,-8.038272e+05,3.285322e+06,-7.115216e+06,-8.038272e+05,3.285322e+06
2,3.019047,172.978645,-0.252186,-14.449184,202.649000,-7.211310e+06,-1.352183e+06,2.830642e+06,-7.211310e+06,-1.352183e+06,2.830642e+06
3,3.230637,185.101892,-0.414670,-23.758859,285.706998,-7.244362e+06,-1.790804e+06,2.442127e+06,-7.244362e+06,-1.790804e+06,2.442127e+06
4,3.710786,212.612386,-0.701128,-40.171673,500.276999,-7.139355e+06,-2.870855e+06,1.379838e+06,-7.139355e+06,-2.870855e+06,1.379838e+06
5,3.904895,223.733979,-0.782428,-44.829838,603.889999,-6.990509e+06,-3.355575e+06,8.454528e+05,-6.990509e+06,-3.355575e+06,8.454528e+05


In [28]:
position_fram_fil

,UT1_hours,alpha,delta,year,month,days,deltaT,Lambda,Miu,Niu,P,Q,SG_d,deltaT_SI,jd
0,21.575128,142.935000,8.521111,1965,1,14,0.000000,-0.789144,0.596067,0.148174,0.296272,-0.351086,78.032128,0.000000,-12769.600293
1,21.603111,157.274167,-2.395000,1965,1,14,0.124857,-0.921558,0.385984,-0.041788,0.520311,-0.193223,78.453011,100.735981,-12769.599128
2,21.631420,171.817917,-14.508333,1965,1,14,0.251173,-0.958257,0.137781,-0.250521,0.708283,-0.008122,78.878811,202.649000,-12769.597948
3,21.654491,183.134167,-23.563333,1965,1,14,0.354119,-0.915248,-0.050115,-0.399763,0.805554,0.131507,79.225833,285.706998,-12769.596987
4,21.714094,208.640417,-40.020278,1965,1,14,0.620067,-0.672115,-0.367065,-0.643059,0.870732,0.370844,80.122323,500.276999,-12769.594503
5,21.742876,219.102083,-44.998611,1965,1,14,0.748489,-0.548745,-0.445986,-0.707090,0.855495,0.434417,80.555226,603.889999,-12769.593304


## 计算残差 见 step3 - echo1st